# HW4P1: Language Modelling

Welcome to the final part 1 hw of this course. This is the only part 1 in which you have PyTorch training (Yay). You will be working on training language models and evaluating them on the task of prediction and generation.<br>
Note: A major change which we have made this semester is that we have made the model which you will be coding in this HW very similar to the Speller module from HW4P2. 

# Get modules and datasets

In [1]:
pip install torchsummaryX


[notice] A new release of pip available: 22.3 -> 23.1
[notice] To update, run: pip3.8 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# TODO: Import drive if you are using Colab

# Imports

In [15]:
import sys
sys.path.append("/Users/guyunxin/Desktop/CMU/23Spring/11685/hw4p1") # TODO: Add path to handout/

In [27]:
sys.path[-1]

'/Users/guyunxin/Desktop/CMU/23Spring/11685/hw4p1'

In [30]:
%matplotlib inline

import torch

import os

import time 
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import torchsummaryX

# Importing necessary modules from hw4
from hw4.tests_hw4 import test_prediction, test_generation

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", DEVICE)

%cd /Users/guyunxin/Desktop/CMU/23Spring/11685/hw4p1/handout 

Device:  cpu
/Users/guyunxin/Desktop/CMU/23Spring/11685/hw4p1/handout


# Load datasets

In [31]:
# Loading the vocabulary. Try printing and see
VOCAB       = np.load('dataset/vocab.npy') 

# We have also included <sos> and <eos> in the vocabulary for you
# However in real life, you include it explicitly if not provided
SOS_TOKEN   = np.where(VOCAB == '<sos>')[0][0]
EOS_TOKEN   = np.where(VOCAB == '<eos>')[0][0]
NUM_WORDS   = len(VOCAB) - 2 # Actual number of words in vocabulary

print("Vocab length: ", len(VOCAB))
print(VOCAB)

Vocab length:  33280
['!' '"' '#' ... '～' '<sos>' '<eos>']


In [62]:
VOCAB[1417]

'<eol>'

In [41]:
SOS_TOKEN

33278

In [42]:
EOS_TOKEN

33279

In [39]:
VOCAB[19]

"'Italia"

In [34]:
SOS_TOKEN 

33278

In [149]:
# Loding the training dataset. Refer to write up section 2 to understand the structure
dataset     = np.load('dataset/wiki.train.npy', allow_pickle=True)

# The dataset does not have <sos> and <eos> because they are just regular articles. 
# TODO: Add <sos> and <eos> to every article in the dataset.
# articles = np.zeros((dataset.shape[0],))
articles = []
for i in range(dataset.shape[0]):
    article = dataset[i]
    article = np.insert(article, 0, SOS_TOKEN)
    article = np.append(article, EOS_TOKEN)
    # articles = np.vstack((articles, article))
    articles.append(article)
# Before doing do, try printing the dataset to see if they are words or integers.


In [150]:
dataset = np.array(articles)

<ipython-input-150-6b293fa8402e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(articles)


In [151]:
# Loading the fixtures for validation and test - prediction
fixtures_pred       = np.load('fixtures/prediction.npz')        # validation
fixtures_pred_test  = np.load('fixtures/prediction_test.npz')   # test

print("Validation shapes    : ", fixtures_pred['inp'].shape, fixtures_pred['out'].shape)
print("Test shapes          : ", fixtures_pred_test['inp'].shape)

Validation shapes    :  (128, 21) (128,)
Test shapes          :  (128, 21)


In [152]:
# Loading the fixtures for validation and test - generation
fixtures_gen        = np.load('fixtures/generation.npy')        # validation
fixtures_gen_test   = np.load('fixtures/generation_test.npy')   # test

print("Validation Gen Shapes    :", fixtures_gen.shape)
print("Test Gen Shapes          :", fixtures_gen_test.shape)

Validation Gen Shapes    : (32, 21)
Test Gen Shapes          : (128, 31)


In [153]:
# Example Prediction Dev Input and Output
# Optional TODO: You can try printing a few samples from the validation set which has both inputs and outputs
print("Input: ", fixtures_pred['inp'][0])
print("Output: ", fixtures_pred['out'][0])
print('generation ', fixtures_gen[0])

Input:  [33278 26096 26972 25821 14658 29325 32935 21820 25639 16134 31353 29092
    79  6916    76 21415 14658 24911  1424 29456 29325]
Output:  72
generation  [33278 32885 31353 22124 32747 20202 28955    76 16786 25881 31453 32846
 32022 14686 31543 15794    79  9916 25821 31365 32846]


In [269]:
dataset[578].shape[0]

2007

# Custom DataLoader

In [270]:
class DataLoaderForLanguageModeling(torch.utils.data.DataLoader): # Inherit from torch.utils.data.DataLoader
    """
        TODO: Define data loader logic here
    """
    # TODO: You can probably add more parameters as well. Eg. sequence length
    def __init__(self, dataset, batch_size, shuffle= False, drop_last= False, sequence_length= 20): 
        
        # If you remember, these are the standard things which you give while defining a dataloader.
        # Now you are just customizing your dataloader
        self.dataset    = dataset
        self.batch_size = batch_size
        self.shuffle    = shuffle
        self.drop_last  = drop_last
        self.sequence_length = sequence_length

    def __len__(self):
        # What output do you get when you print len(loader)? You get the number of batches
        # Your dataset has (579, ) articles and each article has a specified amount of words.
        # You concatenate the dataset and then batch parts of it according to the sequence length
        # TODO: return the number of batches
        # If you are using variable sequence_length, the length might not be fixed 

        concat_length = np.sum([self.dataset[i].shape[0] for i in range(self.dataset.shape[0])])
        # print(concat_length)
        num_of_batches = concat_length//(self.batch_size*self.sequence_length)
        

        return num_of_batches

    def __iter__(self):
        # TODOs: 
        # 1. Shuffle data if shuffle is True
        # 2. Concatenate articles and drop extra words
        # 3. Divide the concatenated dataset into inputs and targets. How do they vary? 
        # 4. Reshape the inputs and targets into batches (think about the final shape)
        # 5. Loop though the batches and yield the input and target according to the sequence length
        
        if self.shuffle:
            # TODO

            np.random.shuffle(self.dataset)
        
        num_batches = self.__len__()
        
        # print(num_batches)

        batches = []
        # concat dataset and create batch
        concat_dataset = np.concatenate(self.dataset)
        for i in range(num_batches):
            batch = concat_dataset[i*self.batch_size*self.sequence_length:(i+1)*self.batch_size*self.sequence_length+1]
            # add extra one sequence to the end 
            # print(batch.shape)
            batches.append(batch)

        batch_idx = 0

        if self.drop_last:
            # TODO
            batches = batches[:-1]

        while batch_idx < num_batches:
            # print(batches[batch_idx].shape)
            inputs = batches[batch_idx][:-1].reshape(self.batch_size, self.sequence_length)
            targets = batches[batch_idx][1:].reshape(self.batch_size, self.sequence_length)
            batch_idx += 1  
            yield torch.tensor(inputs).to(DEVICE), torch.tensor(targets).to(DEVICE)
            

In [271]:
# Some sanity checks

dl = DataLoaderForLanguageModeling(
    dataset     = dataset, 
    batch_size  = 32, 
    shuffle     = True, 
    drop_last   = True
    # Input Extra parameters here if needed
)

inputs, targets = next(iter(dl))
print(inputs.shape, targets.shape)

for x, y in dl:
    print(x.shape, y.shape)
    # shape should be (batch_size, sequence_length)
    print("x: ", [VOCAB[i] for i in x[0, :]])
    print("y: ", [VOCAB[i] for i in y[0, :]])
    break

torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
x:  ['<sos>', '=', 'Curtis', 'Woodhouse', '=', '<eol>', 'Curtis', 'Woodhouse', '(', 'born', '17', 'April', '1980', ')', 'is', 'an', 'English', 'former', 'professional', 'footballer']
y:  ['=', 'Curtis', 'Woodhouse', '=', '<eol>', 'Curtis', 'Woodhouse', '(', 'born', '17', 'April', '1980', ')', 'is', 'an', 'English', 'former', 'professional', 'footballer', 'turned']


# LanguageModel

In [272]:
# Here comes the main portion of this HW.
# You can do this with a regular LSTM similar to HW3P2. 
# However, using LSTMCells will make this Language model very similar to the decoder in HW4P2 and we recommend you use that for writing resuable code.

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, embed_dim, hid_dim): # TODO: Add more parameters if you want
        super().__init__()

        # For all the layers which you will define, please read the documentation thoroughly before implementation

        self.token_embedding    = torch.nn.Embedding(num_embeddings= vocab_size,
                                                     embedding_dim = embed_dim) # TODO: Define a PyTorch embedding layer 

        self.lstm_cells         = torch.nn.Sequential(
            torch.nn.LSTMCell(input_size = embed_dim, hidden_size = hid_dim), # TODO: Enter the parameters for the LSTMCells
            torch.nn.LSTMCell(input_size = hid_dim, hidden_size = hid_dim)
            # You can add multiple LSTMCells too if you want
        )
        self.hidden_dim = hid_dim

        self.token_probability  = torch.nn.Linear(hid_dim, vocab_size) # TODO: Define the parameters

        # Optional TODO: Weight Tying. You just need to make the embedding layer weights equal to the Linear layer weight. 
                
        # So the basic pipline is:
        # word -> embedding -> lstm -> projection (linear) to get  probability distribution
        # And this is happening across all time steps

    def rnn_step(self, embedding, hidden_states_list):
        # initialize hidden and cell states
        hx = torch.zeros((embedding.shape[0],self.hidden_dim)) # (batch, hidden_size)
        cx = torch.zeros((embedding.shape[0],self.hidden_dim))
        hidden_states_list[0]=(hx,cx)
        for i in range(len(self.lstm_cells)):
            # TODO: Forward pass through each LSTMCell
            hx, cx = self.lstm_cells[i](embedding, hidden_states_list[-1])
            hidden_states_list[i]=(hx,cx)
        embedding = hx
        return embedding, hidden_states_list

    def predict(self, x):
        # Refer to Section 1.3.1 to understand this function
        if not torch.is_tensor(x):
            x = torch.tensor(x).long().to(DEVICE)

        with torch.inference_mode():
            # TODO: Pass the input sequence through the model 
            # and return the probability distribution of the last timestep
            prob,_ = self.forward(x)

            return prob

    def generate(self, x, timesteps): 
        # Refer to section 1.3.2 to understand this function
        # Important Note: We do not draw <eos> from the distribution unlike the writeup
        if not torch.is_tensor(x):
            x = torch.tensor(x).long().to(DEVICE)

        # TODO: Pass the input sequence through the model 
        # Obtain the probability distribution and hidden_states_list of the last timestep
        
        token_prob_dist, hidden_states_list     = self.forward(x)
        next_token                              = token_prob_dist.argmax(dim = 1) 
        # TODO: Draw the next predicted token from the probability distribution ()  

        generated_sequence  = [] 
        with torch.inference_mode():
            for t in range(timesteps): # Loop through the timesteps
                #   TODO: Pass the next_token and hidden_states_list through the model
                token_prob_dist, hidden_states_list = self.forward(next_token, hidden_states_list)
                #   TODO: You will get 2 outputs. What is the shape of the probability distribution?
                #   TODO: Get the most probable token for the next timestep
                next_token = token_prob_dist.argmax(dim = 1)

                generated_sequence.append(next_token)
            
            generated_sequence = torch.stack(generated_sequence, dim= 1) # keep last timesteps generated words

        return generated_sequence

    # We are also having a hidden_states_list parameter because you need that in generation
    def forward(self, x, hidden_states_list= None): # train model
        # x (Batch, Seq_len)
        # Note: you dont have to return the sum of log probabilities according to Pseudocode 1 in the writeup
        # However, feel free to calculate and print it if you are curious

        batch_size, timesteps   = x.shape 

        token_prob_distribution = [] # list which will contain probability distributions for all timesteps
        # Initializing the hidden states
        hidden_states_list      = [None]*len(self.lstm_cells) if hidden_states_list == None else hidden_states_list       

        token_embeddings        = self.token_embedding(x)
        # TODO
        # When you get the embeddings of the input x, remember that you get it for all time steps.
        # Embedding is just a linear transformation so you can precompute it for all time steps.

        for t in range(timesteps): # LSTMCell is for just 1 timestep. Hence you need to loop through the total timesteps

            token_embedding_t  = token_embeddings[:, t, :]
            # TODO  

            rnn_out, hidden_states_list = self.rnn_step(token_embedding_t, hidden_states_list)
            # TODO (What should you do with the hidden_states_list?)
            
            token_prob_dist_t   = self.token_probability(rnn_out)
             # TODO

            token_prob_distribution.append(token_prob_dist_t) 

        token_prob_distribution = torch.stack(token_prob_distribution, dim = 1)
        # TODO: Stack along the timesteps dimension

        return token_prob_distribution, hidden_states_list 

# Trainer Class

In [273]:
# Unlike all the P2s, we are using a Trainer class for this HW.
# Many researchers also use classes like this for training. You may have encountered them in your project as well.
# You dont have to complete everything in this class, you only need to complete the train function.
# However, its good to go through the code and see what it does. 

class Trainer:
    def __init__(self, model, loader, optimizer, criterion, scheduler, max_epochs= 1, run_id= 'exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model      = model
        self.loader     = loader
        self.optimizer  = optimizer
        self.criterion  = criterion
        self.scheduler  = scheduler

        self.train_losses           = []
        self.val_losses             = []
        self.predictions            = []
        self.predictions_test       = []
        self.generated_logits       = []
        self.generated              = []
        self.generated_logits_test  = []
        self.generated_test         = []
        self.epochs                 = 0
        self.max_epochs             = max_epochs
        self.run_id                 = run_id


    def calculate_loss(self, out, target):
        # output: (B, T, Vocab_size) - probability distributions
        # target: (B, T)
        # Read the documentation of CrossEntropyLoss and try to understand how it takes inputs

        # Tip: If your target is of shape (B, T) it means that you have B batches with T words. 
        # Tip: What is the total number of words in this batch? 
        # Tip: Crossentropy calculates the loss between a label and its probability distribution.

        out     = out.reshape(-1, out.shape[-1]) # flatten batch and sequence, keep num of classes - vocab size
        # TODO
        targets = target.reshape(-1)
        # TODO
        loss    = self.criterion(out, targets)

        return loss


    def train(self):

        self.model.train() # set to training mode
        self.model.to(DEVICE)
        epoch_loss  = 0
        num_batches = 0
        
        for batch_num, (inputs, targets) in enumerate(tqdm(self.loader)):

            # TODO: Complete the loop. You should be able to complete this without any helper comments after 3 HWs
            # Tip: Mixed precision training
            # For loss calculation, use the calculate_loss function. You need to complete it before using.
            self.optimizer.zero_grad()
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            out,_ = self.model(inputs)
            loss = self.calculate_loss(out, targets)
            loss.backward()
            self.optimizer.step()
            self.scheduler.step(loss)

            num_batches += 1


            loss = loss.item()
            epoch_loss += loss
        
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN] \tEpoch [%d/%d] \tLoss: %.4f \tLr: %.6f'
                      % (self.epochs, self.max_epochs, epoch_loss, self.optimizer.param_groups[0]['lr']))
        self.train_losses.append(epoch_loss)


    
    def test(self): # Don't change this function
        
        self.model.eval() # set to eval mode
        predictions     = self.model.predict(fixtures_pred['inp']).detach().cpu().numpy() # get predictions
        self.predictions.append(predictions)

        generated_logits        = self.model.generate(fixtures_gen, 10).detach().cpu().numpy() # generated predictions for 10 words
        generated_logits_test   = self.model.generate(fixtures_gen_test, 10).detach().cpu().numpy()

        nll             = test_prediction(predictions, fixtures_pred['out'])
        generated       = test_generation(fixtures_gen, generated_logits, VOCAB)
        generated_test  = test_generation(fixtures_gen_test, generated_logits_test, VOCAB)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = self.model.predict(fixtures_pred_test['inp']).detach().cpu().numpy() # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL] \tEpoch [%d/%d] \tLoss: %.4f'
                      % (self.epochs, self.max_epochs, nll))
        return nll

    
    def save(self): # Don't change this function

        model_path = os.path.join('hw4/experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()}, model_path)
        np.save(os.path.join('hw4/experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('hw4/experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('hw4/experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('hw4/experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        
        with open(os.path.join('hw4/experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])

        with open(os.path.join('hw4/experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])

# Experiment setup

In [274]:
# TODO: define other hyperparameters here

configs = dict(
    batch_size  = 256,
    num_epochs  = 10, # 10 or 20 epochs should be enough given the model is good

    init_lr     = 0.001 # TODO
)

In [275]:
model       = LanguageModel(vocab_size = len(VOCAB), embed_dim = 128, hid_dim = 128)
# TODO: Define the model

loader      = DataLoaderForLanguageModeling(dataset, batch_size = 32) # TODO: Define the dataloader

criterion   = torch.nn.CrossEntropyLoss() 

optimizer   = torch.optim.Adam(model.parameters(), lr=configs['init_lr'])
# TODO: Define the optimizer. Adam/AdamW usually works good for this HW

# Optional TODO: Use a scheduler if you want
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', patience =2)

print(model)
torchsummaryX.summary(model, x = inputs.to(DEVICE))

LanguageModel(
  (token_embedding): Embedding(33280, 128)
  (lstm_cells): Sequential(
    (0): LSTMCell(128, 128)
    (1): LSTMCell(128, 128)
  )
  (token_probability): Linear(in_features=128, out_features=33280, bias=True)
)
                          Kernel Shape   Output Shape    Params  Mult-Adds
Layer                                                                     
0_token_embedding         [128, 33280]  [32, 20, 128]  4.25984M    4259840
1_lstm_cells.LSTMCell_0              -      [32, 128]  132.096k     131072
2_lstm_cells.LSTMCell_1              -      [32, 128]  132.096k     131072
3_token_probability       [128, 33280]    [32, 33280]  4.29312M    4259840
4_lstm_cells.LSTMCell_0              -      [32, 128]         -     131072
5_lstm_cells.LSTMCell_1              -      [32, 128]         -     131072
6_token_probability       [128, 33280]    [32, 33280]         -    4259840
7_lstm_cells.LSTMCell_0              -      [32, 128]         -     131072
8_lstm_cells.LSTMCell_1 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_token_embedding,"[128, 33280]","[32, 20, 128]",4259840.0,4259840
1_lstm_cells.LSTMCell_0,-,"[32, 128]",132096.0,131072
2_lstm_cells.LSTMCell_1,-,"[32, 128]",132096.0,131072
3_token_probability,"[128, 33280]","[32, 33280]",4293120.0,4259840
4_lstm_cells.LSTMCell_0,-,"[32, 128]",NaN,131072
...,...,...,...,...
56_lstm_cells.LSTMCell_1,-,"[32, 128]",NaN,131072
57_token_probability,"[128, 33280]","[32, 33280]",NaN,4259840
58_lstm_cells.LSTMCell_0,-,"[32, 128]",NaN,131072


In [276]:
# Dont change this cell

run_id = str(int(time.time()))
if not os.path.exists('./hw4/experiments'):
    os.mkdir('./hw4/experiments')
os.mkdir('./hw4/experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./hw4/experiments/%s" % run_id)

# The object of the Trainer class takes in everything
trainer = Trainer(
    model       = model, 
    loader      = loader, 
    

    optimizer   = optimizer,
    criterion   = criterion, 
    scheduler   = scheduler,
    max_epochs  = configs['num_epochs'], 
    run_id      = run_id
)

Saving models, predictions, and generated words to ./hw4/experiments/1682215502


In [277]:
# Run the experiments loop. 
# Each epoch wont take more than 2-3min. If its taking more time, it might be due to (but not limited to) the following:
#   * You might be overlapping batches 
#       Eg. Input: "I had biryani for lunch today" and sequence length = 3,
#           --> "I had biryani", "for lunch today" are ideal examples for inputs
#           --> "I had biryani", "had biryani for", "biryani for lunch", ... is just redundant info :')
#   * Your length calculation in the dataloader might be wrong
# If you haven't had biryani, try it :D 

# %%time
best_nll = 1e30 
for epoch in range(configs['num_epochs']):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch+1)+" with NLL: "+ str(best_nll))
        trainer.save()    

  0%|          | 0/3245 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses[0:10], label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

# Create handin

In [ ]:
# TODO: Generate the handin to submit to autolab
!make runid=<ENTER RUN ID> epoch=<ENTER EPOCH NUMBER>